In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchinfo import summary
from tqdm import tqdm

In [3]:
# 设置随机数种子，便于复现
torch.manual_seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
# 使用cuDNN加速卷积运算
torch.backends.cudnn.benchmark = True

In [6]:
# 载入训练集
train_dataset = torchvision.datasets.MNIST(
    root="../mnist/",
    train=True,
    transform=transforms.ToTensor(),
    download=False
)

# 载入测试集
test_dataset = torchvision.datasets.MNIST(
    root="../mnist/",
    train=False,
    transform=transforms.ToTensor(),
    download=False
)

# 生成dataloader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)